In [1]:
from z3 import *

In [3]:
def at_least_one(bool_vars):
    return Or(bool_vars)

def at_most_one(bool_vars):
    return [Not(And(pair[0], pair[1])) for pair in combinations(bool_vars, 2)]

def exactly_one(bool_vars):
    return at_most_one(bool_vars) + [at_least_one(bool_vars)]

### Parameters
- $n$ number of blocks
- $B_i$, $i = 0, ..., n-1$ blocks to insert where $B_i = \{x_i, y_i\}$ are the dimensions of the $i$-block, weight and height respectively
- width $w$
- ```rot = True``` if rotation is allowed 

$\huge{\lor_{\small k=0}^{\small n-1}} \huge{\land_{\small r=i}^{\small i+y_k} \land_{\small c=j}^{\small j+x_k}}$ $\Big( \texttt{cells}_{r,c,k} = 1 \land (i+y_k < n) \land (j+x_k<n) \Big)$ $\forall i,j=0,...,n-1 $

In [4]:
def vlsi_sat(n, b, w, rot=False):
    ''' - Each cell assumes a value between 0 and n (included), where the 0-value
            means that no block is inserted in that cell
        - the height of the grid is the sum of the height of the blocks
    '''
    h = sum([bh for (bw, bh) in b])
    # cells[i][j] : cell of row i and column j
    cells = [[[Bool(f"c_{i}_{j}_{k}") for k in range(n + 1)] for j in range(w)] 
         for i in range(h)]
    
    s = Solver()
    # Each cell must contain a value only, corresponding to the block 
    # number e.g. the blocks can't overlap.
    for i in range(h):
        for j in range(w):
            s.add(exactly_one([cells[i][j][k] for k in range(n + 1)]))

    for i in range(h):
        for j in range(w):
            
            for k in range(n):
                for r in range(i, i + b[k][1]):
                    for c in range(j + b[k][0]):
                        cnst += And(exactly_one(cells[r][c][k]), r < n, c < n)
                    cnst = And(cnst)
                cnst(Or(cnst))